# `virtual`: How to virtualize Redset

In [ ]:
%reload_ext autoreload
%autoreload 2

import virtual

# s3_parquet_filepath = 's3://redshift-downloads/redset/serverless/sample_0.01.parquet'
s3_parquet_filepath = '../../demo/data/redset-serverless.parquet'

### One-shot table compression

In [ ]:
# Virtualize Parquet file stored on S3.
virtual.to_format(s3_parquet_filepath, 'redset_virtual.parquet', model_types=['sparse-lr'], prefix='demo-debug/')

### Compare to vanilla `Parquet`

In [ ]:
import os
import demo_util
%pip install requests

print(f'Redset parquet size: {demo_util.get_file_size(s3_parquet_filepath) / 1_000_000} MB')
print(f'Redset virtual size: {os.path.getsize('redset_virtual.parquet') / 1_000_000} MB')

### Query the vanilla Parquet file

In [ ]:
import duckdb

duckdb.sql(f'select num_scans, count(*) from read_parquet("{s3_parquet_filepath}") group by num_scans order by num_scans;').fetchdf()

### Query the virtualized Parquet file

In [ ]:
virtual.query(
  'select num_scans, count(*) from read_parquet("redset_virtual.parquet") group by num_scans order by num_scans',
  engine = 'duckdb'
)